<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-1" data-toc-modified-id="Part-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 1</a></span></li><li><span><a href="#Part-2" data-toc-modified-id="Part-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 2</a></span></li><li><span><a href="#Part-3" data-toc-modified-id="Part-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part 3</a></span></li><li><span><a href="#Part-4" data-toc-modified-id="Part-4-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Part 4</a></span></li></ul></div>

In [98]:
import pyspark

import pandas as pd
from pyspark.sql import SparkSession,\
    DataFrameNaFunctions, DataFrameStatFunctions,\
    functions, types, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.functions import format_string, regexp_extract, trim, upper

import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.master("local").appName("read").\
    enableHiveSupport().\
    getOrCreate()

## Part 1
Read the `case.csv` file from the 311 call data into a Spark DataFrame.

In [3]:
df = spark.read.csv('sa311/case.csv', sep=',', header=True, inferSchema=True)

How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [39]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [45]:
df.select('SLA_due_date', 'case_closed', 'num_days_late')\
    .where(df.case_closed == 'NO')\
    .groupby().max('num_days_late').show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



In [60]:
import datetime
today = datetime.date.today()

df.select('case_opened_date').\
    withColumn('date',\
               to_timestamp(df.case_opened_date, 'M/d/yy')).\
    select(col('date'),\
           datediff(current_timestamp(), col('date'))).show(1)

+-------------------+-----------------------------------+
|               date|datediff(current_timestamp(), date)|
+-------------------+-----------------------------------+
|2018-01-01 00:00:00|                                500|
+-------------------+-----------------------------------+
only showing top 1 row



How many Stray Animal cases are there?

In [70]:
df.select('service_request_type')\
    .where(df.service_request_type == 'Stray Animal')\
    .count()

26760

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [73]:
df.select('dept_division', 'service_request_type')\
    .where(df.dept_division == 'Field Operations')\
    .where(df.service_request_type != 'Officer Standby')\
    .count()

113902

Create a new DataFrame without any information related to dates or location.

In [76]:
no_dates_df = (df.drop('case_opened_date',
                       'case_closed_date',
                       'SLA_due_date',
                       'request_address'))

Read `dept.csv` into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [77]:
df = spark.read.format("csv").\
    option("sep", ",").\
    option("header", True).\
    option("inferSchema", True).\
    load("sa311/dept.csv")

In [78]:
df.fillna('other', 'dept_name')

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

## Part 2

Convert the `council_district` column to a string column.

In [87]:
df = spark.read.format("csv").\
    option("sep", ",").\
    option("header", True).\
    option("inferSchema", True).\
    load("sa311/case.csv")

In [88]:
df.select('council_district', format_string('%010d', 'council_district').\
         alias('council_district_fixed')).show(5)

+----------------+----------------------+
|council_district|council_district_fixed|
+----------------+----------------------+
|               5|            0000000005|
|               3|            0000000003|
|               3|            0000000003|
|               3|            0000000003|
|               7|            0000000007|
+----------------+----------------------+
only showing top 5 rows



Extract the year from the `case_closed_date` column.

In [92]:
df.select('case_closed_date').show(1)

+----------------+
|case_closed_date|
+----------------+
|    1/1/18 12:29|
+----------------+
only showing top 1 row



In [97]:
df.select('case_closed_date', regexp_extract(\
                                             df.case_closed_date.cast('string'),\
                                             '\d+/\d+/(\d+)', 1).alias('year')).show(5)

+----------------+----+
|case_closed_date|year|
+----------------+----+
|    1/1/18 12:29|  18|
|     1/3/18 8:11|  18|
|     1/2/18 7:57|  18|
|     1/2/18 8:13|  18|
|    1/1/18 13:29|  18|
+----------------+----+
only showing top 5 rows



Convert `num_days_late` from days to hours in new columns `num_hours_late`.

In [102]:
df.select('num_days_late', round(df.num_days_late  * 24, 2)).alias('num_hours_late').show(5)

+-------------------+------------------------------+
|      num_days_late|round((num_days_late * 24), 2)|
+-------------------+------------------------------+
| -998.5087616000001|                     -23964.21|
|-2.0126041669999997|                         -48.3|
|       -3.022337963|                        -72.54|
|       -15.01148148|                       -360.28|
|0.37216435200000003|                          8.93|
+-------------------+------------------------------+
only showing top 5 rows



Convert the `case_late` column to a boolean column.

In [106]:
df.select('case_late', (df['case_late'] == 'YES').alias('case_late_boolean')).show(5)

+---------+-----------------+
|case_late|case_late_boolean|
+---------+-----------------+
|       NO|            false|
|       NO|            false|
|       NO|            false|
|       NO|            false|
|      YES|             true|
+---------+-----------------+
only showing top 5 rows



Convert the `SLA_days` columns to a double column.
Pull it all together

In [111]:
df.select('SLA_days', df.SLA_days.cast('double')).show()

+------------------+------------------+
|          SLA_days|          SLA_days|
+------------------+------------------+
|             999.0|             999.0|
|       4.322222222|       4.322222222|
|       4.320729167|       4.320729167|
|       16.29188657|       16.29188657|
|             0.125|             0.125|
|       30.08446759|       30.08446759|
|       16.06429398|       16.06429398|
|16.063796300000003|16.063796300000003|
|       16.06333333|       16.06333333|
|        16.0628588|        16.0628588|
|       16.06237269|       16.06237269|
|       16.06104167|       16.06104167|
|       16.06059028|       16.06059028|
|       16.06011574|       16.06011574|
|       16.05953704|       16.05953704|
|       16.05907407|       16.05907407|
|       16.05864583|       16.05864583|
|       16.05819444|       16.05819444|
|       16.05775463|       16.05775463|
|       16.05733796|       16.05733796|
+------------------+------------------+
only showing top 20 rows



## Part 3

Create a DataFrame with all combinations of `council_district` and `service_request_type` (regardless of whether the combination is observed in the data).

In [115]:
df.groupby('council_district').pivot('service_request_type').count().limit(5).toPandas()

,council_district,"""Vacant Lot-City (12"""")""","""Vacant Lot-City (48"""")""","""Vacant Lot-County (12"""")""","""Vacant Lot-County (48"""")""","""Vacant Lot-Private (12"""")""","""Vacant Lot-Private (48"""")""","""Vacant Lot-State (12"""")""","""Vacant Lot-State (48"""")""",1st Call Bagged Leaves,...,Zoning: District Requirement,Zoning: Illumination,Zoning: Junk Yards,Zoning: Mobile Homes,Zoning: Multi-Family In Single,Zoning: Outside Storage,Zoning: Oversized Vehicles,Zoning: Recycle Yard,Zoning: Setbacks,Zoning: Visual Obstruction
0,1,48,15,2.0,3.0,841,67,73.0,5.0,594,...,362,32,5.0,20,142,51,167,12.0,141,276
1,6,58,7,4.0,1.0,801,9,2.0,1.0,883,...,324,9,20.0,15,41,47,170,39.0,60,153
2,3,100,74,5.0,2.0,824,151,4.0,6.0,203,...,386,22,23.0,65,114,40,233,23.0,89,240
3,5,98,97,4.0,10.0,619,250,1.0,2.0,157,...,181,13,48.0,39,134,33,122,84.0,109,227
4,9,2,1,NaN,NaN,90,1,NaN,NaN,1251,...,59,13,NaN,4,21,11,86,NaN,53,129


Join the case data with the source and department data.

In [116]:
source_df = spark.read.csv('sa311/source.csv', header=True, inferSchema=True)
case_df = spark.read.csv('sa311/case.csv', header=True, inferSchema=True)
dept_df = spark.read.csv('sa311/dept.csv', header=True, inferSchema=True)

In [119]:
full_df = (df.join(source_df, df.source_id == source_df.source_id, 'left')\
    .join(dept_df, df.dept_division == dept_df.dept_division, 'left'))\
    .limit(10)\
    .toPandas()

Are there any cases that do not have a request source?

None

In [123]:
(df.join(source_df, df.source_id == source_df.source_id, 'left')\
    .join(dept_df, df.dept_division == dept_df.dept_division, 'left')).where(f.isnull(df.source_id)).show()

+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+---------+---------------+-------------+---------+----------------------+-------------------+
|case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|source_id|request_address|council_district|source_id|source_username|dept_division|dept_name|standardized_dept_name|dept_subject_to_SLA|
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+---------+---------------+-------------+---------+----------------------+-------------------+
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+---------

## Part 4

Who are the top 10 service request types in terms of number of requests?

In [126]:
full = (df.join(source_df, 
         df.source_id == source_df.source_id, 
         'left')
        .join(dept_df, 
       df.dept_division == dept_df.dept_division, 
       'left'))

In [127]:
full.groupBy(df.service_request_type).count().sort('count', ascending=False).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



Who are the top 10 service request types in terms of average days late?

In [129]:
full.select(df.service_request_type,\
            df.num_days_late)\
    .groupBy(df.service_request_type)\
    .avg('num_days_late')\
    .sort('avg(num_days_late)', ascending=False)\
    .show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420932|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.112556415     |
|Zoning: Recycle Yard                  |135.9285161247979 |
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.87050230311685 |
+--------------------------------------+------------------+
only showing top 10 rows



Does number of days late depend on department?

In [130]:
full.select(full.dept_name, full.num_days_late)\
    .groupBy('dept_name')\
    .avg('num_days_late')\
    .show(10)

+--------------------+-------------------+
|           dept_name| avg(num_days_late)|
+--------------------+-------------------+
|Animal Care Services| -226.5178394055038|
|                null|  135.9285161247979|
|Solid Waste Manag...|-2.2000575136721308|
|Development Services| 13.433724555869683|
|Trans & Cap Impro...| -20.61283735405259|
|    Customer Service| 59.737091496300735|
|        Metro Health| -4.911766979607019|
|Parks and Recreation| -5.251521960055141|
|Code Enforcement ...| -38.70133068329481|
|        City Council|               null|
+--------------------+-------------------+



How do number of days late depend on department division and request type?